## Accelerate Inference: Neural Network Pruning

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, regularizers
from tensorflow.keras.layers import *

print(tf.version.VERSION)

2.9.2


In [3]:
# untar
# !tar -xvzf dataset.tar.gz
!tar -xvzf '/content/drive/MyDrive/dataset.tar.gz'
# load train
train_images = pickle.load(open('train_images.pkl', 'rb'))
train_labels = pickle.load(open('train_labels.pkl', 'rb'))
# load val
val_images = pickle.load(open('val_images.pkl', 'rb'))
val_labels = pickle.load(open('val_labels.pkl', 'rb'))

train_images.pkl
train_labels.pkl
val_images.pkl
val_labels.pkl


In [4]:
# Define the neural network architecture (don't change this)

model = models.Sequential()
model.add(Conv2D(32, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5), input_shape=(25,25,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

In [5]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 25, 25, 32)        896       
                                                                 
 activation (Activation)     (None, 25, 25, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 23, 23, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 23, 23, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 11, 11, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 11, 11, 32)        0         
                                                        

In [6]:
# you can use the default hyper-parameters for training, 
# and val accuracy ~59% after 25 epochs and > 63% after 50 epochs

model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4, decay=1e-6),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, batch_size=32, epochs=50, 
                   validation_data=(val_images, val_labels)) # train for 50 epochs, with batch size 32

Epoch 1/50


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


703/703 [==============================] - 8s 7ms/step - loss: 1.4751 - accuracy: 0.3344 - val_loss: 1.3371 - val_accuracy: 0.4309
Epoch 2/50
703/703 [==============================] - 4s 6ms/step - loss: 1.3318 - accuracy: 0.4275 - val_loss: 1.2590 - val_accuracy: 0.4570
Epoch 3/50
703/703 [==============================] - 4s 6ms/step - loss: 1.2672 - accuracy: 0.4655 - val_loss: 1.1972 - val_accuracy: 0.5065
Epoch 4/50
703/703 [==============================] - 4s 6ms/step - loss: 1.2226 - accuracy: 0.4911 - val_loss: 1.1599 - val_accuracy: 0.5204
Epoch 5/50
703/703 [==============================] - 4s 6ms/step - loss: 1.1905 - accuracy: 0.5073 - val_loss: 1.1296 - val_accuracy: 0.5343
Epoch 6/50
703/703 [==============================] - 4s 6ms/step - loss: 1.1593 - accuracy: 0.5292 - val_loss: 1.0966 - val_accuracy: 0.5541
Epoch 7/50
703/703 [==============================] - 4s 6ms/step - loss: 1.1274 - accuracy: 0.5390 - val_loss: 1.0843 - val_accuracy: 0.5529
Epoch 8/50
703/70

In [7]:
results = model.evaluate(val_images, val_labels, batch_size=128)

20/20 [==============================] - 0s 10ms/step - loss: 0.7303 - accuracy: 0.7196


In [8]:
def sparsity(weights):
  zero_weights = 0
  total_weights = 0
  for i in range(len(weights)):
    curr_zero_weights = tf.reduce_sum((weights[i]==0)*1).numpy()
    # print(curr_zero_weights)
    zero_weights += curr_zero_weights
    total_weights += weights[i].reshape(-1).shape[0]
  return zero_weights / total_weights

def score(acc, sparsity):
  return (acc + sparsity)/2

def magnitude_based_pruning(weights, threshold):
  for i in range(len(weights)):
    weights[i] = (abs(weights[i]) > threshold)  * weights[i]
  return weights

In [9]:
thr = 0.05
loop = 20
epoch_per_train = 5
for i in range(loop):
  print()
  print('i: {}/{}'.format(i,loop))
  history = model.fit(train_images, train_labels, batch_size=32, epochs=epoch_per_train, 
                    validation_data=(val_images, val_labels), verbose=False)
  weights = model.get_weights()
  new_weights = magnitude_based_pruning(weights, threshold = thr)
  model.set_weights(new_weights)

  results = model.evaluate(val_images, val_labels, batch_size=128, verbose=False)
  sparsity_model = sparsity(model.get_weights())
  score_model  = score(results[1], sparsity_model)
  print('Accuracy: {:.3f} || Sparsity: {:.3f} || Score: {:.3f}'.format(results[1],sparsity_model,score_model))



i: 0/20
Accuracy: 0.576 || Sparsity: 0.686 || Score: 0.631

i: 1/20
Accuracy: 0.666 || Sparsity: 0.702 || Score: 0.684

i: 2/20
Accuracy: 0.700 || Sparsity: 0.713 || Score: 0.707

i: 3/20
Accuracy: 0.714 || Sparsity: 0.722 || Score: 0.718

i: 4/20
Accuracy: 0.705 || Sparsity: 0.729 || Score: 0.717

i: 5/20
Accuracy: 0.718 || Sparsity: 0.736 || Score: 0.727

i: 6/20
Accuracy: 0.719 || Sparsity: 0.741 || Score: 0.730

i: 7/20
Accuracy: 0.719 || Sparsity: 0.747 || Score: 0.733

i: 8/20
Accuracy: 0.727 || Sparsity: 0.751 || Score: 0.739

i: 9/20
Accuracy: 0.730 || Sparsity: 0.756 || Score: 0.743

i: 10/20
Accuracy: 0.733 || Sparsity: 0.760 || Score: 0.747

i: 11/20
Accuracy: 0.738 || Sparsity: 0.764 || Score: 0.751

i: 12/20
Accuracy: 0.735 || Sparsity: 0.767 || Score: 0.751

i: 13/20
Accuracy: 0.743 || Sparsity: 0.770 || Score: 0.757

i: 14/20
Accuracy: 0.734 || Sparsity: 0.773 || Score: 0.754

i: 15/20
Accuracy: 0.734 || Sparsity: 0.776 || Score: 0.755

i: 16/20
Accuracy: 0.738 || Spars

In [10]:
# evaluate again to see how the accuracy changes
results = model.evaluate(val_images, val_labels, batch_size=128)
sparsity_model = sparsity(model.get_weights())
score_model  = score(results[1], sparsity_model)

print('Sparsity of the model: {:.12f}'.format(sparsity_model))
print('Score of the model: {:6f}'.format(score_model))
# assert np.less(0.60, results[1]), 'Not getting desired accuracy'
# assert np.less(0.36, score_model), 'Not getting desired score'
# assert np.less(0, sparsity_model), 'No sparsity in the model'

20/20 [==============================] - 0s 5ms/step - loss: 0.7011 - accuracy: 0.7497
Sparsity of the model: 0.786662236711
Score of the model: 0.768183


In [11]:
# you need to save the model's weights, naming it 'my_model_weights.h5'
model_name = "my_model_weights_1.h5"
model.save_weights(model_name)

# running this cell will immediately download a file called 'my_model_weights.h5'
from google.colab import files
files.download(model_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>